In [31]:
from base_layer_utils import BaseLayerDataRepo, BaseLayerResultsRepo, ModelName
from base_layer_utils import LightgbmBLE, SklearnBLE, XGBoostBLE

#from fast_text_data import fasttext_data_process
#from tfidf_data import tfidf_data_process

import pandas as pd
import numpy as np
#from sklearn.cross_validation import KFold # replace with model_selection?
#from sklearn.model_selection import KFold
import time, re, gc
import xgboost as xgb
from sklearn.metrics import roc_auc_score

In [32]:
PATH = '~/data/toxic/data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

print(train.shape)
print(test.shape)

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

(159571, 27)
(153164, 21)


# select layer2 outputs

In [42]:
base_layer_results_repo = BaseLayerResultsRepo()

load from file


In [43]:
_ = base_layer_results_repo.show_scores()

0.9827	ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9826	ModelName.NBSVM_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9819	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9818	ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9815	ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9803	ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9794	ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9793	ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9786	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w
0.9774	ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000
0.9768	ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000
0.9765	ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000
0.9761	ModelName.NBSVM_tfidf_word_df2_ng(1, 2)_wmf200000
0.976	ModelName.LOGREG_tfidf_word_df2_ng(1, 1)_wmf200000
0.9752	ModelName.LOGR

base_layer_results_repo.remove('ModelName.LGB_layer2')

In [35]:
selected = ['ModelName.XGB_layer2', 'ModelName.LOGREG_layer2']

In [36]:
layer2_oof_train_loaded, layer2_oof_test_loaded, layer2_est_preds_loaded = base_layer_results_repo.get_results(chosen_ones=selected)

In [37]:
assert len(layer2_oof_train_loaded['toxic']) == len(layer2_est_preds_loaded) == len(selected)

### before we choose which models to assemble, we can do:
#### 1. scatter plot analysis to check the diversity
#### 2. submit to check if the models have similar performance

## Ensembling:

In [38]:
def combine_layer_oof_per_label(layer1_oof_dict, label):
    x = None
    data_list = layer1_oof_dict[label]
    for i in range(len(data_list)):
        if i == 0:
            x = data_list[0]
        else:
            x = np.concatenate((x, data_list[i]), axis=1)
    return x

### 1. simple blend of two models

In [39]:
result = np.empty((test.shape[0],len(label_cols)))

# mix the first two models
for i, label in enumerate(label_cols):
    x_train = combine_layer_oof_per_label(layer2_oof_train_loaded, label)
    x_test = combine_layer_oof_per_label(layer2_oof_test_loaded, label)
    for j in range(x_train.shape[1]):
        roc = roc_auc_score(train[label], x_train[:,j])
        print(label, j, roc) # print out roc for meta feature on meta label (which is just the original train label)
    
    roc_scores_of_a_label = []
    alphas = np.linspace(0,1,1001)
    best_roc = 0
    best_alpha = 0
    for alpha in alphas:
        roc = roc_auc_score(train[label], alpha*x_train[:,0] + (1-alpha)*x_train[:,1])
        if roc > best_roc:
            best_roc = roc
            best_alpha = alpha
    
    print(label, best_roc, best_alpha)
    result[:,i] = best_alpha*x_test[:,0] + (1-best_alpha)*x_test[:,1]

toxic 0 0.9910814282997946
toxic 1 0.990771077509489
toxic 0.9910856078589341 0.9550000000000001
severe_toxic 0 0.9925828223897508
severe_toxic 1 0.9919322096940085
severe_toxic 0.9928764823290488 0.128
obscene 0 0.9962181243081879
obscene 1 0.9959665500533149
obscene 0.9962206810311406 0.984
threat 0 0.9946855970430916
threat 1 0.99310288682231
threat 0.9948137226753532 0.294
insult 0 0.991952202527335
insult 1 0.991528941423655
insult 0.9919546286846996 0.936
identity_hate 0 0.9933174155555204
identity_hate 1 0.9925218214135401
identity_hate 0.9934864460389672 0.203


In [41]:
submission = pd.read_csv(PATH + 'sample_submission.csv')#.head(1000)
submission[label_cols] = result
sub_id = int(time.time())
print(sub_id)
submission.to_csv('./StackPreds/layer3_mix_9854xgb_9853logreg_' + str(sub_id) + '.csv', index=False)

1520889439


### 2. stacking

In [12]:
base_layer_results_repo = BaseLayerResultsRepo()

load from file


base_layer_results_repo.add_score('ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real',0.9819)

In [ ]:
scores = base_layer_results_repo.show_scores()

In [15]:
layer1_oof_train_loaded, layer1_oof_test_loaded, base_layer_est_preds_loaded = base_layer_results_repo.get_results(threshold=0.9793)

In [10]:
ths = [score for _, score in scores][3:10]

In [11]:
ths

[0.9815, 0.9803, 0.9794, 0.9793, 0.9786, 0.9774, 0.9768]

In [29]:
#base_layer_results_repo.add_score('ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real',0.9818)

ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real already existed in the repo. score: 0 update to 0.9818


f = open('./xgb_search.csv', 'a')
# header = 'time,id,th,amt,csbt,lr,md,ss,ga,a,rounds,folds,tops,sps,ops,\
# thps,inps,ihps,tobr,sbr,obr,thbr,inbr,ihbr,sth1,sth2,sth3,sth4,sth5,\
# to_auc,s_auc,o_auc,th_auc,in_auc,ih_auc,avg_auc\n'
header = 'time,id,threshold,num_models,colsample_bytree,lr,max_depth,subsample,\
gamma,alpha,cv_num_round,cv_nfolds,toxic_pos_scale,severe_toxic_pos_scale,obscene_pos_scale,\
threat_pos_scale,insult_pos_scale,identity_hate_pos_scale,sth1,sth2,sth3,sth4,sth5,\
toxic_best_round,severe_toxic_best_round,obscene_best_round,threat_best_round,\
insult_best_round,identity_hate_best_round,toxic_auc,severe_toxic_auc,obscene_auc,\
threat_auc,insult_auc,identity_hate_auc,avg_auc\n'
f.write(header)
f.close()

In [9]:
def get_time():
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')


    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)
    
    return est.strftime('%Y-%m-%d %H:%M:%S')

In [4]:
lgb_stacker_params = {
    'max_depth':3, 
    'metric':"auc", 
    'n_estimators':125, 
    'num_leaves':10, 
    'boosting_type':"gbdt", 
    'learning_rate':0.1, 
    'feature_fraction':0.45, 
    'colsample_bytree':0.45, 
    'bagging_fraction':0.8, 
    'bagging_freq':5, 
    'reg_lambda':0.2
}

In [5]:
lgb_stacker = LightgbmBLE(None, None, params=lgb_stacker_params, nb=False, seed=1001)

Naive Bayes is disabled
LightgbmBLE is initialized


In [71]:
xgb_stacker = XGBoostBLE(None, None, params={}, nb=False, seed=1001)

Naive Bayes is disabled
XGBoostBase is initialized


In [72]:
model_pool = {}
model_pool[ModelName.LGB] = lgb_stacker
model_pool[ModelName.XGB] = xgb_stacker

layer1_oof_train_loaded, layer1_oof_test_loaded, base_layer_est_preds_loaded

In [85]:
layer2_inputs = {}
# let lGB use half of the base layer model+data, and XGB use another half
layer2_inputs[ModelName.LGB] = base_layer_results_repo.get_results(chosen_ones=selected[:int(len(selected)/2)])
layer2_inputs[ModelName.XGB] = base_layer_results_repo.get_results(chosen_ones=selected[int(len(selected)/2):])

In [78]:
# import lightgbm as lgb
# stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, 
#                              boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, 
#                              colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)

layer2_est_preds = {} # directly preditions from the base layer estimators

layer2_oof_train = {}
layer2_oof_test = {}

model_data_id_list = []

# result = np.empty((test.shape[0],len(label_cols)))
for model_name in model_pool.keys():
    for i, label in enumerate(label_cols):
        assert train.shape == (159571, 27)

        model = model_pool[model_name]
        if model_name == ModelName.XGB: # Done some grid search on xgb, so it has per label based best params
            model.set_params(get_best_xgb_params(label=label))

        x_train = combine_layer_oof_per_label(layer1_oof_train_loaded, label)
        x_test = combine_layer_oof_per_label(layer1_oof_test_loaded, label)

        # add engineered features to layer 2
        x_train = np.hstack([F_train[features].as_matrix(), x_train])
        x_test = np.hstack([F_test[features].as_matrix(), x_test])  

        SEED = 1001
        NFOLDS = 2 # set folds for out-of-fold prediction

        oof_train, oof_test = get_oof(model,  x_train, train[label], x_test, NFOLDS, SEED)

        if label not in layer2_oof_train:
            layer2_oof_train[label] = []
            layer2_oof_test[label] = []
        layer2_oof_train[label].append(oof_train)
        layer2_oof_test[label].append(oof_test)

    #     stacker.fit(x_train, train[label])
    #     result[:,i] = stacker.predict_proba(x_test)[:,1]
        model_data_id = '{}_{}'.format(model_name, 'layer2')
        model.train(x_train, train[label])
        est_preds = model.predict(x_test)

        if model_data_id not in layer2_est_preds:
            layer2_est_preds[model_data_id] = np.empty((x_test.shape[0],len(label_cols)))
            model_data_id_list.append(model_data_id)
        layer2_est_preds[model_data_id][:,i] = est_preds

No evaluation set, thus not possible to use early stopping. Please train with your best params.
starting predicting
predicting done
starting predicting
predicting done
No evaluation set, thus not possible to use early stopping. Please train with your best params.
starting predicting
predicting done
starting predicting
predicting done
No evaluation set, thus not possible to use early stopping. Please train with your best params.
starting predicting
predicting done
No evaluation set, thus not possible to use early stopping. Please train with your best params.
starting predicting
predicting done
starting predicting
predicting done
No evaluation set, thus not possible to use early stopping. Please train with your best params.
starting predicting
predicting done
starting predicting
predicting done
No evaluation set, thus not possible to use early stopping. Please train with your best params.
starting predicting
predicting done
No evaluation set, thus not possible to use early stopping. Plea

In [79]:
len(layer2_oof_train['toxic'])

2

In [80]:
layer2_oof_train['toxic'][0].shape

(159571, 1)

In [81]:
list(layer2_est_preds)

['ModelName.XGB_layer2', 'ModelName.LGB_layer2']

In [82]:
layer2_est_preds['ModelName.LGB_layer2'].shape

(153164, 6)

In [86]:
list(model_data_id_list)

['ModelName.LGB_layer2', 'ModelName.XGB_layer2']

base_layer_results_repo.add(layer2_oof_train, layer2_oof_test, layer2_est_preds, model_data_id_list)

In [91]:
base_layer_results_repo.show_scores()

0.9827	ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9826	ModelName.NBSVM_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9819	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9818	ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9815	ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9803	ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9794	ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9793	ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9786	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w
0.9774	ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000
0.9768	ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000
0.9765	ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000
0.9761	ModelName.NBSVM_tfidf_word_df2_ng(1, 2)_wmf200000
0.976	ModelName.LOGREG_tfidf_word_df2_ng(1, 1)_wmf200000
0.9752	ModelName.LOGR

[('ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real', 0.9827),
 ('ModelName.NBSVM_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real', 0.9826),
 ('ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real', 0.9819),
 ('ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real',
  0.9818),
 ('ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
  0.9815),
 ('ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
  0.9803),
 ('ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
  0.9794),
 ('ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
  0.9793),
 ('ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w', 0.9786),
 ('ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000', 0.9774),
 ('ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000', 0.9768),
 ('ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000', 0.9765),
 ('ModelName.NBSVM_tfidf_word_df2_ng(1, 2

In [84]:
sub_title = 'xgb_topn_w_addedfeatures_'
submission = pd.read_csv(PATH + 'sample_submission.csv')
submission[label_cols] = layer2_est_preds['ModelName.XGB_layer2']
tempid = int(time.time())
print(tempid)
submission.to_csv('./StackPreds/TopN_XGB/{}_{}.csv'.format(sub_title, tempid), index=False)

1520883170


In [66]:
def get_best_xgb_params(xgb_grid_search_res_path='~/data/kaggle/toxic/sc/stacking/xgb_search.csv', idx = 0, label='toxic'):
    """
    Params: 
        xgb_grid_search_res_path: (str).
        idx: load the nth best params. e.g. idx=0, load the best. idx=1, load the 2nd best
    Returns:
        (dict) the best xgb grid search params.
    """       
    xgb_search = pd.read_csv(xgb_grid_search_res_path).sort_values(by='avg_auc', ascending=False)
    
    print('xgb grid search file loaded. shape:{}'.format(xgb_search.shape))
    
    i=idx

    search_id = xgb_search['id'].values[i]
    model_threshold = xgb_search['threshold'].values[i]
    num_models = xgb_search['num_models'].values[i]
    xgb_colsample_bytree = xgb_search['colsample_bytree'].values[i]
    xgb_learning_rate = xgb_search['lr'].values[i]
    xgb_max_depth = xgb_search['max_depth'].values[i]
    xgb_subsample = xgb_search['subsample'].values[i]
    xgb_gamma = xgb_search['gamma'].values[i]
    xgb_alpha = xgb_search['alpha'].values[i]
    xgb_cv_num_round = xgb_search['cv_num_round'].values[i]
    xgb_cv_nfolds = xgb_search['cv_nfolds'].values[i]
    
    # per label based params 
    best_nrounds = {}
    best_nrounds['toxic'] = xgb_search['toxic_best_round'].values[i]
    best_nrounds['severe_toxic'] = xgb_search['severe_toxic_best_round'].values[i]
    best_nrounds['obscene'] = xgb_search['obscene_best_round'].values[i]
    best_nrounds['threat'] = xgb_search['threat_best_round'].values[i]
    best_nrounds['insult'] = xgb_search['insult_best_round'].values[i]
    best_nrounds['identity_hate'] = xgb_search['identity_hate_best_round'].values[i]
    
    scale_pos_weights = {}
    scale_pos_weights['toxic'] = xgb_search['toxic_pos_scale'].values[i]
    scale_pos_weights['severe_toxic'] = xgb_search['severe_toxic_pos_scale'].values[i]
    scale_pos_weights['obscene'] = xgb_search['obscene_pos_scale'].values[i]
    scale_pos_weights['threat'] = xgb_search['threat_pos_scale'].values[i]
    scale_pos_weights['insult'] = xgb_search['insult_pos_scale'].values[i]
    scale_pos_weights['identity_hate'] = xgb_search['identity_hate_pos_scale'].values[i]

    metric_dict_fromcsv = {}
    metric_dict_fromcsv['avg_auc'] = xgb_search['avg_auc'].values[i]
    
    xgb_params = {
        'seed': 0,
        'colsample_bytree': xgb_colsample_bytree,
        'silent': 1,
        'subsample': xgb_subsample,
        'learning_rate': xgb_learning_rate,
        'max_depth': xgb_max_depth,
        'gamma': xgb_gamma,
        'alpha': xgb_alpha,
        'nthread': 7,
        'min_child_weight': 1,
        'objective':'binary:logistic',
        'eval_metric':'auc'
    }

    xgb_params['scale_pos_weight'] = scale_pos_weights[label]
    xgb_params['num_boost_round'] = best_nrounds[label]
    
    return xgb_params

In [69]:
get_best_xgb_params(label='threat')

xgb grid search file loaded. shape:(81, 36)


{'alpha': 0,
 'colsample_bytree': 0.9,
 'eval_metric': 'auc',
 'gamma': 2,
 'learning_rate': 0.0966631,
 'max_depth': 3,
 'min_child_weight': 1,
 'nthread': 7,
 'num_boost_round': 119,
 'objective': 'binary:logistic',
 'scale_pos_weight': 120,
 'seed': 0,
 'silent': 1,
 'subsample': 0.9}

In [74]:
isinstance({}, dict)

True

In [76]:
from sklearn.model_selection import KFold, StratifiedKFold

def get_oof(clf, x_train, y_train, x_test, nfolds, stratified=False, shuffle=True, seed=1001):
    #pdb.set_trace()
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((nfolds, ntest))
    if stratified:
        kf = StratifiedKFold(n_splits=nfolds, shuffle=shuffle, random_state=seed)
    else:
        kf = KFold(n_splits=nfolds, shuffle=shuffle, random_state=seed)

    for i, (tr_index, te_index) in enumerate(kf.split(x_train, y_train)):
        x_tr, x_te = x_train[tr_index], x_train[te_index]
        y_tr, y_te = y_train.iloc[tr_index], y_train.iloc[te_index]
        
        clf.train(x_tr, y_tr)

        oof_train[te_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
}